In [5]:
import os
import warnings
from functools import lru_cache
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pynverse import inversefunc

from numpy import pi
from scipy.fftpack import fft, ifft
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib  import cm
import pickle
from scipy.interpolate import interp2d

matplotlib.use("Agg")

%matplotlib inline

> Discretize and numerically solve the Kuramoto–Sivashinsky equation, using the same parameters as [Vlachas et. al](https://doi.org/10.1016/j.neunet.2020.02.016), section 5.3. Option: Fourier transform. Solution should be similar to figure 14

![](images/kuramoto.png)

### Equation:

<center>$\large \frac{\partial u}{\partial t}=-v\frac{\partial^4 u}{\partial x^4}-\frac{\partial^2 u}{\partial x^2}-u\frac{\partial u}{\partial x}$</center>

Using fft method implement by Vlachas in [pvlachas/RNN-RC-Chaos](https://github.com/pvlachas/RNN-RC-Chaos)

In [6]:
# code from: https://github.com/pvlachas/RNN-RC-Chaos/blob/master/Data/KuramotoSivashinskyGP512/Utils/KS.py

np.seterr(over='raise', invalid='raise')

class KS:
    #
    # Solution of the 1D Kuramoto-Sivashinsky equation
    #
    # u_t + u*u_x + u_xx + u_xxxx = 0,
    # with periodic BCs on x \in [0, 2*pi*L]: u(x+2*pi*L,t) = u(x,t).
    #
    # The nature of the solution depends on the system size L and on the initial
    # condition u(x,0).  Energy enters the system at long wavelengths via u_xx
    # (an unstable diffusion term), cascades to short wavelengths due to the
    # nonlinearity u*u_x, and dissipates via diffusion with u_xxxx.
    #
    # Spatial  discretization: spectral (Fourier)
    # Temporal discretization: exponential time differencing fourth-order Runge-Kutta
    # see AK Kassam and LN Trefethen, SISC 2005

    def __init__(self, L=16, N=128, dt=0.25, nsteps=None, tend=150, iout=1):
        #
        # Initialize
        L  = float(L); dt = float(dt); tend = float(tend)
        if (nsteps is None):
            nsteps = int(tend/dt)
        else:
            nsteps = int(nsteps)
            # override tend
            tend = dt*nsteps
        #
        # save to self
        self.L      = L
        self.N      = N
        self.dx     = 2*pi*L/N
        self.dt     = dt
        self.nsteps = nsteps
        self.iout   = iout
        self.nout   = int(nsteps/iout)
        #
        # set initial condition
        self.IC()
        #
        # initialize simulation arrays
        self.setup_timeseries()
        #
        # precompute Fourier-related quantities
        self.setup_fourier()
        #
        # precompute ETDRK4 scalar quantities:
        self.setup_etdrk4()


    def setup_timeseries(self, nout=None):
        if (nout != None):
            self.nout = int(nout)
        # nout+1 so we store the IC as well
        self.vv = np.zeros([self.nout+1, self.N], dtype=np.complex64)
        self.tt = np.zeros(self.nout+1)
        #
        # store the IC in [0]
        self.vv[0,:] = self.v0
        self.tt[0]   = 0.


    def setup_fourier(self, coeffs=None):
        self.x  = 2*pi*self.L*np.r_[0:self.N]/self.N
        self.k  = np.r_[0:self.N/2, 0, -self.N/2+1:0]/self.L # Wave numbers
        # Fourier multipliers for the linear term Lu
        if (coeffs is None):
            # normal-form equation
            self.l = self.k**2 - self.k**4
        else:
            # altered-coefficients 
            self.l = -      coeffs[0]*np.ones(self.k.shape) \
                     -      coeffs[1]*1j*self.k             \
                     + (1 + coeffs[2])  *self.k**2          \
                     +      coeffs[3]*1j*self.k**3          \
                     - (1 + coeffs[4])  *self.k**4


    def setup_etdrk4(self):
        self.E  = np.exp(self.dt*self.l)
        self.E2 = np.exp(self.dt*self.l/2.)
        self.M  = 16                                           # no. of points for complex means
        self.r  = np.exp(1j*pi*(np.r_[1:self.M+1]-0.5)/self.M) # roots of unity
        self.LR = self.dt*np.repeat(self.l[:,np.newaxis], self.M, axis=1) + np.repeat(self.r[np.newaxis,:], self.N, axis=0)
        self.Q  = self.dt*np.real(np.mean((np.exp(self.LR/2.) - 1.)/self.LR, 1))
        self.f1 = self.dt*np.real( np.mean( (-4. -    self.LR              + np.exp(self.LR)*( 4. - 3.*self.LR + self.LR**2) )/(self.LR**3) , 1) )
        self.f2 = self.dt*np.real( np.mean( ( 2. +    self.LR              + np.exp(self.LR)*(-2. +    self.LR             ) )/(self.LR**3) , 1) )
        self.f3 = self.dt*np.real( np.mean( (-4. - 3.*self.LR - self.LR**2 + np.exp(self.LR)*( 4. -    self.LR             ) )/(self.LR**3) , 1) )
        self.g  = -0.5j*self.k


    def IC(self, u0=None, v0=None, testing=False):
        #
        # Set initial condition, either provided by user or by "template"
        if (v0 is None):
            # IC provided in u0 or use template
            if (u0 is None):
                # set u0
                if testing:
                    # template from AK Kassam and LN Trefethen, SISC 2005
                    u0 = np.cos(self.x/self.L)*(1. + np.sin(self.x/self.L))
                else:
                    # random noise
                    u0 = (np.random.rand(self.N) -0.5)*0.01
            else:
                # check the input size
                if (np.size(u0,0) != self.N):
                    print('Error: wrong IC array size')
                    return -1
                else:
                    # if ok cast to np.array
                    u0 = np.array(u0)
            # in any case, set v0:
            v0 = fft(u0)
        else:
            # the initial condition is provided in v0
            # check the input size
            if (np.size(v0,0) != self.N):
                print('Error: wrong IC array size')
                return -1
            else:
                # if ok cast to np.array
                v0 = np.array(v0)
                # and transform to physical space
                u0 = ifft(v0)
        #
        # and save to self
        self.u0  = u0
        self.v0  = v0
        self.v   = v0
        self.t   = 0.
        self.stepnum = 0
        self.ioutnum = 0 # [0] is the initial condition
        

    def step(self):
        #
        # Computation is based on v = fft(u), so linear term is diagonal.
        # The time-discretization is done via ETDRK4
        # (exponential time differencing - 4th order Runge Kutta)
        #
        v = self.v;                           Nv = self.g*fft(np.real(ifft(v))**2)
        a = self.E2*v + self.Q*Nv;            Na = self.g*fft(np.real(ifft(a))**2)
        b = self.E2*v + self.Q*Na;            Nb = self.g*fft(np.real(ifft(b))**2)
        c = self.E2*a + self.Q*(2.*Nb - Nv);  Nc = self.g*fft(np.real(ifft(c))**2)
        #
        self.v = self.E*v + Nv*self.f1 + 2.*(Na + Nb)*self.f2 + Nc*self.f3
        self.stepnum += 1
        self.t       += self.dt


    def simulate(self, nsteps=None, iout=None, restart=False, correction=[]):
        #
        # If not provided explicitly, get internal values
        if (nsteps is None):
            nsteps = self.nsteps
        else:
            nsteps = int(nsteps)
            self.nsteps = nsteps
        if (iout is None):
            iout = self.iout
            nout = self.nout
        else:
            self.iout = iout
        if restart:
            # update nout in case nsteps or iout were changed
            nout      = int(nsteps/iout)
            self.nout = nout
            # reset simulation arrays with possibly updated size
            self.setup_timeseries(nout=self.nout)
        #
        # advance in time for nsteps steps
        if (correction==[]):
            for n in range(1,self.nsteps+1):
                try:
                    self.step()
                except FloatingPointError:
                    #
                    # something exploded
                    # cut time series to last saved solution and return
                    self.nout = self.ioutnum
                    self.vv.resize((self.nout+1,self.N)) # nout+1 because the IC is in [0]
                    self.tt.resize(self.nout+1)          # nout+1 because the IC is in [0]
                    return -1
                if ( (self.iout>0) and (n%self.iout==0) ):
                    self.ioutnum += 1
                    self.vv[self.ioutnum,:] = self.v
                    self.tt[self.ioutnum]   = self.t
        else:
            # lots of code duplication here, but should improve speed instead of having the 'if correction' at every time step
            for n in range(1,self.nsteps+1):
                try:
                    self.step()
                    self.v += correction
                except FloatingPointError:
                    #
                    # something exploded
                    # cut time series to last saved solution and return
                    self.nout = self.ioutnum
                    self.vv.resize((self.nout+1,self.N)) # nout+1 because the IC is in [0]
                    self.tt.resize(self.nout+1)          # nout+1 because the IC is in [0]
                    return -1
                if ( (self.iout>0) and (n%self.iout==0) ):
                    self.ioutnum += 1
                    self.vv[self.ioutnum,:] = self.v
                    self.tt[self.ioutnum]   = self.t


    def fou2real(self):
        #
        # Convert from spectral to physical space
        self.uu = np.real(ifft(self.vv))


    def compute_Ek(self):
        #
        # compute all forms of kinetic energy
        #
        # Kinetic energy as a function of wavenumber and time
        self.compute_Ek_kt()
        # Time-averaged energy spectrum as a function of wavenumber
        self.Ek_k = np.sum(self.Ek_kt, 0)/(self.ioutnum+1) # not self.nout because we might not be at the end; ioutnum+1 because the IC is in [0]
        # Total kinetic energy as a function of time
        self.Ek_t = np.sum(self.Ek_kt, 1)
		# Time-cumulative average as a function of wavenumber and time
        self.Ek_ktt = np.cumsum(self.Ek_kt, 0) / np.arange(1,self.ioutnum+2)[:,None] # not self.nout because we might not be at the end; ioutnum+1 because the IC is in [0] +1 more because we divide starting from 1, not zero
		# Time-cumulative average as a function of time
        self.Ek_tt = np.cumsum(self.Ek_t, 0) / np.arange(1,self.ioutnum+2)[:,None] # not self.nout because we might not be at the end; ioutnum+1 because the IC is in [0] +1 more because we divide starting from 1, not zero

    def compute_Ek_kt(self):
        try:
            self.Ek_kt = 1./2.*np.real( self.vv.conj()*self.vv / self.N ) * self.dx
        except FloatingPointError:
            #
            # probable overflow because the simulation exploded, try removing the last solution
            problem=True
            remove=1
            self.Ek_kt = np.zeros([self.nout+1, self.N]) + 1e-313
            while problem:
                try:
                    self.Ek_kt[0:self.nout+1-remove,:] = 1./2.*np.real( self.vv[0:self.nout+1-remove].conj()*self.vv[0:self.nout+1-remove] / self.N ) * self.dx
                    problem=False
                except FloatingPointError:
                    remove+=1
                    problem=True
        return self.Ek_kt


    def space_filter(self, k_cut=2):
        #
        # spatially filter the time series
        self.uu_filt  = np.zeros([self.nout+1, self.N])
        for n in range(self.nout+1):
            v_filt = np.copy(self.vv[n,:])    # copy vv[n,:] (otherwise python treats it as reference and overwrites vv on the next line)
            v_filt[np.abs(self.k)>=k_cut] = 0 # set to zero wavenumbers > k_cut
            self.uu_filt[n,:] = np.real(ifft(v_filt))
        #
        # compute u_resid
        self.uu_resid = self.uu - self.uu_filt


    def space_filter_int(self, k_cut=2, N_int=10):
        #
        # spatially filter the time series
        self.N_int        = N_int
        self.uu_filt      = np.zeros([self.nout+1, self.N])
        self.uu_filt_int  = np.zeros([self.nout+1, self.N_int])
        self.x_int        = 2*pi*self.L*np.r_[0:self.N_int]/self.N_int
        for n in range(self.nout+1):
            v_filt = np.copy(self.vv[n,:])   # copy vv[n,:] (otherwise python treats it as reference and overwrites vv on the next line)
            v_filt[np.abs(self.k)>=k_cut] = 313e6
            v_filt_int = v_filt[v_filt != 313e6] * self.N_int/self.N
            self.uu_filt_int[n,:] = np.real(ifft(v_filt_int))
            v_filt[np.abs(self.k)>=k_cut] = 0
            self.uu_filt[n,:] = np.real(ifft(v_filt))
        #
        # compute u_resid
        self.uu_resid = self.uu - self.uu_filt

In [7]:
# code from: https://github.com/pvlachas/RNN-RC-Chaos/blob/master/Data/KuramotoSivashinskyGP512/0_data_generation.py

#------------------------------------------------------------------------------
# define data and initialize simulation
L    = 100/(2*pi)
N    = 512
dt   = 0.25
ninittransients = 10000
tend = 50000 + ninittransients  #50000
dns  = KS(L=L, N=N, dt=dt, tend=tend)


N_data_train = 100000
N_data_test = 100000
dl_max = 20000
pl_max = 20000


#------------------------------------------------------------------------------
# simulate initial transient
dns.simulate()
# convert to physical space
dns.fou2real()


u = dns.uu[ninittransients:]

print(u.shape)

N_train_max = int(u.shape[0])
print('Max N_train:')
print(N_train_max)

[u_train, u_test, _] = np.split(u, [N_data_train, N_data_train+N_data_test], axis=0)
print("Traing data shape: ")
print(u_test.shape)
print(u_train.shape)

train_input_sequence = u_train
test_input_sequence = u_test


max_idx = np.shape(test_input_sequence)[0] - pl_max
min_idx = dl_max
idx = np.arange(min_idx, max_idx)
np.random.shuffle(idx)
testing_ic_indexes = idx

attractor_std = np.std(train_input_sequence, axis=0)
print(np.shape(attractor_std))

print(train_input_sequence.shape)
data = {"train_input_sequence":train_input_sequence,
        "attractor_std":attractor_std, "dt":dt}

with open("./Data/training_data_N{:d}.pickle".format(N_data_train), "wb") as file:
    # Pickle the "data" dictionary using the highest protocol available.
    pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)

del data

data = {"test_input_sequence":test_input_sequence,
        "testing_ic_indexes":testing_ic_indexes,
        "attractor_std":attractor_std, "dt":dt}
print(test_input_sequence.shape)

with open("./Data/testing_data_N{:d}.pickle".format(N_data_train), "wb") as file:
    # Pickle the "data" dictionary using the highest protocol available.
    pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)
    
del data



data = {"dns":dns,
        "L":L,
        "N":N, "dt":dt}
print(test_input_sequence.shape)

with open("./Data/simulation_data.pickle", "wb") as file:
    # Pickle the "data" dictionary using the highest protocol available.
    pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)
    
del data

(230001, 512)
Max N_train:
230001
Traing data shape: 
(100000, 512)
(100000, 512)
(512,)
(100000, 512)
(100000, 512)
(100000, 512)


In [8]:
ks_data = pd.concat([pd.DataFrame(u_train), pd.DataFrame(u_test)])

In [9]:
ks_data.columns = ks_data.columns.map(str)

In [10]:
ks_data.reset_index(drop=True).to_parquet('Data/KS_data.parquet')

In [11]:
# https://github.com/pvlachas/RNN-RC-Chaos/blob/master/Data/KuramotoSivashinskyGP512/1_creating_figures.py

base_path = "."

with open(base_path + "/Data/simulation_data.pickle", "rb") as file:
    # Pickle the "data" dictionary using the highest protocol available.
    data = pickle.load(file)
    u = data["dns"].uu[10000:]
    dt = data["dns"].dt
    N = data["N"]
    del data


for N_plot in [1000, 10000, 100000]:
    u_plot = u[:N_plot,:]
    N_plot = np.shape(u_plot)[0]
    # Plotting the contour plot
    fig = plt.subplots()
    # t, s = np.meshgrid(np.arange(N_plot)*dt, np.array(range(N))+1)
    n, s = np.meshgrid(np.arange(N_plot), np.array(range(N))+1)
    plt.contourf(s, n, np.transpose(u_plot), 15, cmap=plt.get_cmap("seismic"))
    plt.colorbar()
    plt.xlabel(r"$x$")
    plt.ylabel(r"$n, \quad t=n \cdot {:}$".format(dt))
    plt.savefig(base_path + "/Figures/Plot_U_first_N{:d}.png".format(N_plot), bbox_inches="tight")
    plt.close()

for N_plot in [1000, 10000, 100000]:
    u_plot = u[-N_plot:,:]
    N_plot = np.shape(u_plot)[0]
    # Plotting the contour plot
    fig = plt.subplots()
    # t, s = np.meshgrid(np.arange(N_plot)*dt, np.array(range(N))+1)
    n, s = np.meshgrid(np.arange(N_plot), np.array(range(N))+1)
    plt.contourf(s, n, np.transpose(u_plot), 15, cmap=plt.get_cmap("seismic"))
    plt.colorbar()
    plt.xlabel(r"$x$")
    plt.ylabel(r"$n, \quad t=n \cdot {:}$".format(dt))
    plt.savefig(base_path + "/Figures/Plot_U_last_N{:d}.png".format(N_plot), bbox_inches="tight")
    plt.close()

In [12]:
figs = [f'Figures/{i}' for i in os.listdir('Figures')]

In [13]:
figs

['Figures/Plot_U_first_N1000.png',
 'Figures/Plot_U_first_N10000.png',
 'Figures/Plot_U_first_N100000.png',
 'Figures/Plot_U_last_N1000.png',
 'Figures/Plot_U_last_N10000.png',
 'Figures/Plot_U_last_N100000.png']

![](Figures/Plot_U_last_N10000.png)
![](Figures/Plot_U_first_N100000.png)
![](Figures/Plot_U_first_N1000.png)
![](Figures/Plot_U_last_N100000.png)
![](Figures/Plot_U_first_N10000.png)
![](Figures/Plot_U_last_N1000.png)